In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from xgboost import XGBClassifier
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.decomposition import PCA
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import TruncatedSVD

In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SG/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SG/test.csv')
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SG/sample_submission.csv')
ids = test['Complaint-ID']

In [6]:
train.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,07/07/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,07/07/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,05/07/2015,Bank account or service,Using a debit or ATM card,NaN,05/07/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...


In [7]:
test.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Te-1,8/18/2016,Bank account or service,"Account opening, closing, or management",Company has responded to the consumer and the ...,8/18/2016,No,XXXX / XXXX / 16 I called Citibank to open a c...
1,Te-2,4/18/2016,Debt collection,Communication tactics,Company believes it acted appropriately as aut...,4/20/2016,No,I'm struggling financially. I called and I off...
2,Te-3,3/23/2016,Credit reporting,Incorrect information on credit report,NaN,3/23/2016,No,"In XXXX of 2015, an automatic payment was conf..."
3,Te-4,6/26/2017,Student loan,Dealing with your lender or servicer,NaN,6/26/2017,NaN,"I submitted a request to XXXX, which is my cur..."
4,Te-5,5/13/2016,Credit reporting,Incorrect information on credit report,Company has responded to the consumer and the ...,5/13/2016,No,A state tax lien was filed against me XXXX / X...


In [8]:
submission.head()

,Complaint-ID,Complaint-Status
0,Te-1,Closed with explanation
1,Te-2,Closed with explanation
2,Te-3,Closed with explanation
3,Te-4,Closed with explanation
4,Te-5,Closed with explanation


In [0]:
train['Consumer-disputes'] = train['Consumer-disputes'].fillna('Yes')
test['Consumer-disputes'] = test['Consumer-disputes'].fillna('Yes')

In [0]:
train['Company-response'] = train['Company-response'].fillna(' ')
test['Company-response'] = test['Company-response'].fillna(' ')

In [11]:
train.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,07/07/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,07/07/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,05/07/2015,Bank account or service,Using a debit or ATM card,,05/07/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...


In [12]:
test.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Te-1,8/18/2016,Bank account or service,"Account opening, closing, or management",Company has responded to the consumer and the ...,8/18/2016,No,XXXX / XXXX / 16 I called Citibank to open a c...
1,Te-2,4/18/2016,Debt collection,Communication tactics,Company believes it acted appropriately as aut...,4/20/2016,No,I'm struggling financially. I called and I off...
2,Te-3,3/23/2016,Credit reporting,Incorrect information on credit report,,3/23/2016,No,"In XXXX of 2015, an automatic payment was conf..."
3,Te-4,6/26/2017,Student loan,Dealing with your lender or servicer,,6/26/2017,Yes,"I submitted a request to XXXX, which is my cur..."
4,Te-5,5/13/2016,Credit reporting,Incorrect information on credit report,Company has responded to the consumer and the ...,5/13/2016,No,A state tax lien was filed against me XXXX / X...


In [0]:
stop = []
stop_en = set(stopwords.words('english'))
stop_sp = set(stopwords.words('spanish'))
stop_fr = set(stopwords.words('french'))
stop.extend(stop_en)
stop.extend(stop_sp)
stop.extend(stop_fr)
stop = [token.lower() for token in stop]

In [0]:
train_sum = train['Consumer-complaint-summary']

train_sum = train_sum.apply(lambda x: x.replace('X', ''))
train_sum = train_sum.apply(lambda x: x.replace('Y', ''))
train_sum = train_sum.apply(lambda x: x.replace('   ', ' '))
train_sum = train_sum.apply(lambda x: x.replace('  ', ' '))
train_sum = train_sum.apply(lambda x: x.replace('&lt;', ''))
train_sum = train_sum.apply(lambda x: x.replace('&gt;', ''))
train_sum = train_sum.apply(lambda x: x.replace('<[^<]+?>', ''))
train_sum = train_sum.apply(lambda x: x.replace('\r\n', ''))
train_sum = train_sum.apply(lambda x: x.replace('\r', ''))
train_sum = train_sum.apply(lambda x: x.replace(r"[^a-zA-Z]+", ''))
train_sum = train_sum.apply(lambda x: x.replace(('('), ''))
train_sum = train_sum.apply(lambda x: x.replace((')'), ''))
train_sum = train_sum.apply(lambda x: x.replace(('{'), ''))
train_sum = train_sum.apply(lambda x: x.replace(('}'), ''))
train_sum = train_sum.apply(lambda x: x.replace((','), ''))
train_sum = train_sum.apply(lambda x: x.replace(('.'), ''))
train_sum = train_sum.apply(lambda x: x.replace((':'), ''))
train_sum = train_sum.apply(lambda x: re.sub(r'\b\d+\b', '', x))
train_sum = train_sum.apply(lambda x: ' '.join([item.lower() for item in x.split() if item.lower() not in stop]))
train_sum = train_sum.apply(lambda x: ' '.join([item for item in x.split() if len(item)>2]))

In [15]:
train_sum[3]

'loan paid years moved moved months paying debt representative contacted suddenly indicating debt paid owe explained debt paid many years passed destroyed documents need keep since account paid fact owed loan right cash check possession received account paid something asked provide could provide anything despite letter conversation phone continually harassed nothing sent contacted collection letter trying collect debt two additional years total years paid debt tries collect debt paid full receiving letter cascade capital trying collect seems collection agencies problem finding contacted phone account understand spent many years without contact suddenly want collect debt paid entirety somehow records disaster maybe since cash base company representative may stolen money check issued understanding payment made due date check processed never made never done received canceled account unfortunately try destroy records find need keep unfortunately saved knew would happen'

In [0]:
test_sum = test['Consumer-complaint-summary']

test_sum = test_sum.apply(lambda x: x.replace('X', ''))
test_sum = test_sum.apply(lambda x: x.replace('Y', ''))
test_sum = test_sum.apply(lambda x: x.replace('   ', ' '))
test_sum = test_sum.apply(lambda x: x.replace('  ', ' '))
test_sum = test_sum.apply(lambda x: x.replace('&lt;', ''))
test_sum = test_sum.apply(lambda x: x.replace('&gt;', ''))
test_sum = test_sum.apply(lambda x: x.replace('<[^<]+?>', ''))
test_sum = test_sum.apply(lambda x: x.replace('\r\n', ''))
test_sum = test_sum.apply(lambda x: x.replace('\r', ''))
test_sum = test_sum.apply(lambda x: x.replace(r"[^a-zA-Z]+", ''))
test_sum = test_sum.apply(lambda x: x.replace(('('), ''))
test_sum = test_sum.apply(lambda x: x.replace((')'), ''))
test_sum = test_sum.apply(lambda x: x.replace(('{'), ''))
test_sum = test_sum.apply(lambda x: x.replace(('}'), ''))
test_sum = test_sum.apply(lambda x: x.replace((','), ''))
test_sum = test_sum.apply(lambda x: x.replace(('.'), ''))
test_sum = test_sum.apply(lambda x: x.replace((':'), ''))
test_sum = test_sum.apply(lambda x: re.sub(r'\b\d+\b', '', x))
test_sum = test_sum.apply(lambda x: ' '.join([item.lower() for item in x.split() if item.lower() not in stop]))
test_sum = test_sum.apply(lambda x: ' '.join([item for item in x.split() if len(item)>2]))

In [17]:
test_sum[2]

'automatic payment configured capital line personal account amount incorrectly configured first statement reflected partial month payments made day month amount instead correct amount another mortgage making additional payment per month last two years reduce capital lender never notified lack payment phone text message email even though information provided husband office never sent late payment letter mail due failure lender notify properly several mortgages businesses lines credit credit cards never delayed last years credit score dropped almost points line closed lender given fair chance remedy error problem would solved immediately eliminate late payment credit agencies involuntary error amount automatic payment payments made every month extra payment made lender another meeting totally unfair harmful consumers pay years deserve ruined credit lender could made sure contact'

In [0]:
train['Consumer-complaint-summary'] = train_sum
test['Consumer-complaint-summary'] = test_sum

In [0]:
train_df = pd.DataFrame(columns=['Text'])
test_df = pd.DataFrame(columns=['Text'])
train_df['Text'] = train['Transaction-Type'] + ' ' + train['Complaint-reason'] + ' ' + train['Company-response'] + ' ' + train['Consumer-disputes'] + ' ' + train['Consumer-complaint-summary']
test_df['Text'] = test['Transaction-Type'] + ' ' + test['Complaint-reason'] + ' ' + test['Company-response'] + ' ' + test['Consumer-disputes'] + ' ' + test['Consumer-complaint-summary']

In [0]:
train_df['Text'] = train_df['Text'].apply(lambda x: ' '.join([item.lower() for item in x.split() if item.lower() not in stop]))
train_df['Text'] = train_df['Text'].apply(lambda x: ' '.join([item for item in x.split() if len(item)>2]))
train_df['Text'] = train_df['Text'].apply(lambda x: x.replace((','), ''))
train_df['Text'] = train_df['Text'].apply(lambda x: x.replace(('.'), ''))

test_df['Text'] = test_df['Text'].apply(lambda x: ' '.join([item.lower() for item in x.split() if item.lower() not in stop]))
test_df['Text'] = test_df['Text'].apply(lambda x: ' '.join([item for item in x.split() if len(item)>2]))
test_df['Text'] = test_df['Text'].apply(lambda x: x.replace((','), ''))
test_df['Text'] = test_df['Text'].apply(lambda x: x.replace(('.'), ''))

In [21]:
train_df['Text'][3]

"debt collection cont'd attempts collect debt owed company believes acted appropriately authorized contract law loan paid years moved moved months paying debt representative contacted suddenly indicating debt paid owe explained debt paid many years passed destroyed documents need keep since account paid fact owed loan right cash check possession received account paid something asked provide could provide anything despite letter conversation phone continually harassed nothing sent contacted collection letter trying collect debt two additional years total years paid debt tries collect debt paid full receiving letter cascade capital trying collect seems collection agencies problem finding contacted phone account understand spent many years without contact suddenly want collect debt paid entirety somehow records disaster maybe since cash base company representative may stolen money check issued understanding payment made due date check processed never made never done received canceled acco

In [22]:
test_df['Text'][2]

'credit reporting incorrect information credit report automatic payment configured capital line personal account amount incorrectly configured first statement reflected partial month payments made day month amount instead correct amount another mortgage making additional payment per month last two years reduce capital lender never notified lack payment phone text message email even though information provided husband office never sent late payment letter mail due failure lender notify properly several mortgages businesses lines credit credit cards never delayed last years credit score dropped almost points line closed lender given fair chance remedy error problem would solved immediately eliminate late payment credit agencies involuntary error amount automatic payment payments made every month extra payment made lender another meeting totally unfair harmful consumers pay years deserve ruined credit lender could made sure contact'

In [0]:
to_num ={'Closed with explanation':0, 
'Closed with non-monetary relief':1,
'Closed':2,
'Closed with monetary relief':3,
'Untimely response':4}
to_label ={0:'Closed with explanation', 
1:'Closed with non-monetary relief',
2:'Closed',
3:'Closed with monetary relief',
4:'Untimely response'}
train['Complaint-Status'] = train['Complaint-Status'].map(to_num)

In [0]:
traindf = train_df['Text']
testdf = test_df['Text']

In [0]:
vectorizer = TfidfVectorizer(ngram_range=(1,5))

trainc = vectorizer.fit_transform(traindf)
testc = vectorizer.transform(testdf)

In [0]:
X_train = trainc
y_train = train['Complaint-Status']
X_test = testc

In [27]:
print(X_train.shape)
print(len(y_train))
print(X_test.shape)

(43266, 11586966)
43266
(18543, 11586966)


In [28]:
y_train[:5]

0    0
1    1
2    0
3    0
4    0
Name: Complaint-Status, dtype: int64

In [29]:
# model = XGBClassifier(n_estimators=200, max_depth=12)
model = LinearSVC(random_state=0, penalty='l1', dual=False, C=1.1)
print(model)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

LinearSVC(C=1.1, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=0, tol=0.0001,
     verbose=0)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [30]:
df = pd.DataFrame()
df['y_pred'] = y_pred
df['y_pred'].value_counts()

0    16346
3     1140
1     1025
2       29
4        3
Name: y_pred, dtype: int64

In [31]:
y_pred_label = df['y_pred'].map(to_label)
y_pred_label[:5]

0    Closed with non-monetary relief
1            Closed with explanation
2            Closed with explanation
3            Closed with explanation
4            Closed with explanation
Name: y_pred, dtype: object

In [0]:
submit = pd.DataFrame(columns=['Complaint-ID', 'Complaint-Status'])
submit['Complaint-ID'] = ids
submit['Complaint-Status'] = y_pred_label
submit.to_csv('/content/drive/My Drive/Colab Notebooks/SG/submit.csv', index=False)